# Análisis Temporal de COVID-19 (V2)

Este cuaderno:
- Muestra la conversión de fechas y la agregación mensual.
- Genera los artefactos necesarios ejecutando el pipeline.
- Presenta un gráfico interactivo (Plotly) basado en los 3 países seleccionados.
- Incluye respuestas fundamentadas al análisis.


## 0) Preparación

In [8]:
import pandas as pd
import plotly.express as px
from pathlib import Path
from IPython.display import display, Markdown
# Rutas
BASE = Path('.')
ANALISIS = BASE / 'analisis'
CSV_INPUT = BASE / 'dataset_covid.csv'
CSV_MENSUAL = BASE / 'casos_mensuales_por_pais.csv'
CSV_ORDENADO = BASE / 'dataset_covid_ordenado.csv'
CSV_TENDENCIAS = ANALISIS / 'tendencias_paises.csv'
JSON_TOTAL_MESES = ANALISIS / 'total_mensual_por_pais.json'


## 1) Ejecutar el pipeline para generar archivos (opcional si ya existen)

In [9]:
# Ejecuta el pipeline que ya tienes en temporal_analysis.py
from temporal_analysis import CovidTemporalAnalysis
pipe = CovidTemporalAnalysis()
pipe.run_all()
# Al finalizar, se habrán generado: 
# - casos_mensuales_por_pais.csv
# - dataset_covid_ordenado.csv
# - analisis/tendencias_paises.csv
# - analisis/total_mensual_por_pais.json




PASO 1: Convertir la columna de fecha a datetime

Fechas no parseadas (NaT): 0
Pais                     object
Fecha            datetime64[ns]
Casos Diarios           float64
dtype: object
                       Pais      Fecha  Casos Diarios
0               Afghanistan 2020-01-09            0.0
195772             Honduras 2020-01-09            0.0
301539              Myanmar 2020-01-09            0.0
16272   Antigua and Barbuda 2020-01-09            0.0
120006             Dominica 2020-01-09            0.0


PASO 2: Agregar columna de Mes (primer día del mes) y agrupar por País y Mes

Resumen mensual por país (muestra):
           Pais        Mes  Casos Mensuales
0   Afghanistan 2020-01-01         0.000000
1   Afghanistan 2020-02-01         0.014082
2   Afghanistan 2020-03-01         2.031321
3   Afghanistan 2020-04-01        34.060712
4   Afghanistan 2020-05-01       270.190339
5   Afghanistan 2020-06-01       441.374023
6   Afghanistan 2020-07-01       145.892503
7   Afghanistan 2

## 2) Conversión de fechas y vista previa

In [10]:
df = pd.read_csv(CSV_INPUT)
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
display(Markdown('**Tipos de datos**'))
display(df.dtypes.to_frame('dtype'))
display(Markdown('**Primeras filas ordenadas por fecha**'))
display(df.sort_values('Fecha').head())


**Tipos de datos**

,dtype
Pais,object
Fecha,datetime64[ns]
Casos Diarios,float64


**Primeras filas ordenadas por fecha**

,Pais,Fecha,Casos Diarios
0,Afghanistan,2020-01-09,0.0
195772,Honduras,2020-01-09,0.0
301539,Myanmar,2020-01-09,0.0
16272,Antigua and Barbuda,2020-01-09,0.0
120006,Dominica,2020-01-09,0.0


## 3) Agrupación mensual por país

In [11]:
df['Mes'] = df['Fecha'].dt.to_period('M').dt.to_timestamp()
mensual = (
    df.groupby(['Pais','Mes'], as_index=False)['Casos Diarios']
      .sum()
      .rename(columns={'Casos Diarios':'Casos Mensuales'})
)
display(mensual.head(12))
mensual.to_csv(CSV_MENSUAL, index=False)
print('Mensual guardado en:', CSV_MENSUAL)


,Pais,Mes,Casos Mensuales
0,Afghanistan,2020-01-01,0.000000
1,Afghanistan,2020-02-01,0.014082
2,Afghanistan,2020-03-01,2.031321
3,Afghanistan,2020-04-01,34.060712
4,Afghanistan,2020-05-01,270.190339
5,Afghanistan,2020-06-01,441.374023
6,Afghanistan,2020-07-01,145.892503
7,Afghanistan,2020-08-01,45.481172
8,Afghanistan,2020-09-01,27.023258
9,Afghanistan,2020-10-01,45.428365


Mensual guardado en: casos_mensuales_por_pais.csv


## 4) Tendencias de los 3 países seleccionados (archivo generado por pipeline)

In [12]:
tend = pd.read_csv(CSV_TENDENCIAS, parse_dates=['Mes'])
paises_sel = tend['Pais'].unique().tolist()
display(Markdown('**Países seleccionados:** ' + ', '.join(paises_sel)))
display(tend.head(18))


**Países seleccionados:** Angola, Brunei, El Salvador

,Pais,Mes,Casos Mensuales,MM3
0,Angola,2020-01-01,0.000000,0.000000
1,Angola,2020-02-01,0.000000,0.000000
2,Angola,2020-03-01,0.020045,0.006682
3,Angola,2020-04-01,0.709576,0.243207
4,Angola,2020-05-01,1.359018,0.696213
5,Angola,2020-06-01,4.778611,2.282401
6,Angola,2020-07-01,20.653861,8.930496
7,Angola,2020-08-01,40.221309,21.884594
8,Angola,2020-09-01,62.627060,41.167410
9,Angola,2020-10-01,145.535155,82.794508


## 5) Agregación por mes calendario (solo países seleccionados) y JSON

In [13]:
tmp = tend[['Pais','Mes','Casos Mensuales']].copy()
tmp['MesNum'] = tmp['Mes'].dt.month
total_mes = (
    tmp.groupby(['Pais','MesNum'], as_index=False)['Casos Mensuales']
       .sum()
       .rename(columns={'Casos Mensuales':'Casos Totales'})
)
nombres = {1:'Enero',2:'Febrero',3:'Marzo',4:'Abril',5:'Mayo',6:'Junio',7:'Julio',8:'Agosto',9:'Septiembre',10:'Octubre',11:'Noviembre',12:'Diciembre'}
total_mes['MesNombre'] = total_mes['MesNum'].map(nombres)
total_mes = total_mes.sort_values(['Pais','MesNum']).reset_index(drop=True)
display(total_mes.head(24))
ANALISIS.mkdir(exist_ok=True)
total_mes.to_json(JSON_TOTAL_MESES, orient='records', force_ascii=False)
print('JSON guardado en:', JSON_TOTAL_MESES)


,Pais,MesNum,Casos Totales,MesNombre
0,Angola,1,751.729176,Enero
1,Angola,2,56.489432,Febrero
2,Angola,3,53.053803,Marzo
3,Angola,4,115.789105,Abril
4,Angola,5,246.258805,Mayo
5,Angola,6,184.766264,Junio
6,Angola,7,158.624216,Julio
7,Angola,8,191.112354,Agosto
8,Angola,9,300.615502,Septiembre
9,Angola,10,409.340922,Octubre


JSON guardado en: analisis\total_mensual_por_pais.json


## 6) Gráfico interactivo (Plotly) desde el JSON

In [14]:
df_json = pd.read_json(JSON_TOTAL_MESES, orient='records')
if 'MesNombre' not in df_json.columns:
    df_json['MesNombre'] = df_json['MesNum'].map(nombres)
df_json = df_json.sort_values(['Pais','MesNum'])
fig = px.line(
    df_json, x='MesNombre', y='Casos Totales', color='Pais', markers=True,
    title='Evolución mensual de casos por país (solo países seleccionados)'
)
fig.update_traces(mode='lines+markers', hovertemplate='<b>%{legendgroup}</b><br>Mes=%{x}<br>Casos Totales=%{y:,.2f}<extra></extra>')
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f', xaxis_title='Mes', yaxis_title='Casos Totales', legend_title='País')
fig.show()


## 7) Respuestas fundamentadas

- **[tendencias]** Se observan picos mensuales en distintos meses según el país (ver curvas).
- **[estacionalidad]** Al agregar por `MesNum`, comparamos la estacionalidad independientemente del año.
- **[suavizado]** La columna `MM3` en `tendencias_paises.csv` (promedio móvil 3 meses) apoya la interpretación reduciendo ruido.
- **[comparabilidad]** Formato con miles y 2 decimales y hover unificado facilitan comparar magnitudes; si hay órdenes de magnitud distintos, considerar eje Y logarítmico.
